In [1]:
import pandas as pd
import sys
print(sys.version)

#Python 3.7.4 (default, Aug 9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
#pygbif 0.3.0

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]


In [2]:
## Define the country of interest
country  = 'BE'

## Define the year from where to consider records for the occupancy
year = 2000

## Create a SQLite database with APSW

In [3]:
import sqlite3
from sqlite3 import Error

In [4]:
database = r"..\data\cube.db"
try:
    conn = sqlite3.connect(database)
except Error as e:
    print(e)

In [5]:

 
sql_create_cube_table = """ CREATE TABLE IF NOT EXISTS cube (
                                        year integer,
                                        eea_cell_code text NOT NULL,
                                        taxonKey integer,
                                        n integer,
                                        min_coord_uncertainty float
                                    ); """
 

In [6]:
# create table
try:
    conn.execute(sql_create_cube_table)
except Error as e:
    print(e)

## Clear down the table before filling it up again

In [7]:
conn.execute('DELETE FROM cube')
conn.commit()

In [8]:
with open(r"D:\Documents\notebooks\batty\data\be_species_cube.csv", "r", encoding='utf-8') as f:  
    rows = f.readlines()
    for row in rows:
        fields = row.split(',')
        conn.execute(f'INSERT INTO cube(year, eea_cell_code, taxonKey, n, min_coord_uncertainty)'\
                     f"VALUES ('{fields[0]}','{fields[1]}','{fields[2]}','{fields[3]}','{fields[4]}')") 
conn.commit()

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Documents\\notebooks\\batty\\data\\be_species_cube.csv'

In [ ]:
pd.read_sql_query('SELECT count(year) FROM cube',conn)

In [ ]:
#conn.execute('DROP INDEX allIndex')
conn.execute('CREATE INDEX allIndex ON cube(taxonKey, year, eea_cell_code)')
conn.execute('CREATE INDEX allCellCode ON cube(eea_cell_code)')

In [ ]:
pd.read_sql_query('EXPLAIN QUERY PLAN SELECT taxonKey FROM cube WHERE year > 2000 and taxonKey = 8427624   GROUP BY eea_cell_code', conn)